In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 50.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=8dd797cf89652773064c991becad6f7a483276fe7e32661431fe503cdd66c850
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


Questions
1. Find all the tweets by user
2. Find how many tweets each user has
3. Find all the persons mentioned on tweets
4. Count how many times each person is mentioned
5. Find the 10 most mentioned persons
6. Find all the hashtags mentioned on a tweet
7. Count how many times each hashtag is mentioned
8. Find the 10 most popular Hashtags
9. Find the top 5 countries which tweet the most.

In [2]:
from pyspark.sql import SparkSession , functions as F
spark = SparkSession.builder.appName('twitter').getOrCreate()

In [32]:
from pyspark.sql.types import ArrayType, StringType

In [11]:
df_twitter = spark.read.format('json').load('sample_data/tweets.json')
df_twitter.printSchema()
df_twitter.show(5)

root
 |-- country: string (nullable = true)
 |-- id: string (nullable = true)
 |-- place: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user: string (nullable = true)

+-------------+------------------+---------+--------------------+------------------+
|      country|                id|    place|                text|              user|
+-------------+------------------+---------+--------------------+------------------+
|        India|572692378957430785|   Orissa|@always_nidhi @Yo...|   Srkian_nishu :)|
|United States|572575240615796737|Manhattan|@OnlyDancers Bell...|TagineDiningGlobal|
|United States|572575243883036672|Claremont|1/ "Without the a...|       Daniel Beer|
|United States|572575252020109313|   Vienna|idk why people ha...|  someone actually|
|United States|572575274539356160|   Boston|Taste of Iceland!...|    BostonAttitude|
+-------------+------------------+---------+--------------------+------------------+
only showing top 5 rows



##1. Find all the tweets by user

In [12]:
user = 'BostonAttitude'
#user = input('Enter user name to see their tweets: ')
df_twitter.filter(df_twitter['user']==user).show()

+-------------+------------------+------+--------------------+--------------+
|      country|                id| place|                text|          user|
+-------------+------------------+------+--------------------+--------------+
|United States|572575274539356160|Boston|Taste of Iceland!...|BostonAttitude|
+-------------+------------------+------+--------------------+--------------+



##2. Find how many tweets each user has

In [40]:
df_twitter.groupBy('user').count().orderBy('count', ascending=False).show(5)

+--------------------+-----+
|                user|count|
+--------------------+-----+
|       #QuissyUpSoon|  258|
|Inès Mendes Askiip ♥|  185|
|           #4Rentinc|  100|
|                  MV|   58|
|    williampriceking|   46|
+--------------------+-----+
only showing top 5 rows



##3.Find all the persons mentioned on tweets

In [34]:
def generate_usermentioned(text):
    return [item.lstrip('@') for item in text.split(' ') if item.startswith('@')]
user_mentioned = df_twitter.withColumn('users_mentioned', F.udf(lambda text: generate_usermentioned(text), ArrayType(StringType()))('text'))
user_mentioned.show()

+--------------------+------------------+-----------------+--------------------+-------------------+--------------------+
|             country|                id|            place|                text|               user|     users_mentioned|
+--------------------+------------------+-----------------+--------------------+-------------------+--------------------+
|               India|572692378957430785|           Orissa|@always_nidhi @Yo...|    Srkian_nishu :)|[always_nidhi, Yo...|
|       United States|572575240615796737|        Manhattan|@OnlyDancers Bell...| TagineDiningGlobal|       [OnlyDancers]|
|       United States|572575243883036672|        Claremont|1/ "Without the a...|        Daniel Beer|                  []|
|       United States|572575252020109313|           Vienna|idk why people ha...|   someone actually|                  []|
|       United States|572575274539356160|           Boston|Taste of Iceland!...|     BostonAttitude|    [IcelandNatural]|
|       United States|57

##4. Count how many times each person is mentioned

In [38]:
new_usermentioned_df= user_mentioned.select(F.explode('users_mentioned').alias('users_mentioned'))
new_usermentioned_df = new_usermentioned_df.filter(new_usermentioned_df['users_mentioned'] != '')
new_usermentioned_df.groupBy('users_mentioned').count().show(truncate=False)

+---------------+-----+
|users_mentioned|count|
+---------------+-----+
|DjRockyUg      |1    |
|TrillHD        |1    |
|TimmysWell     |1    |
|brookie_baldwin|1    |
|TTTorrez       |2    |
|boytoyjesse    |1    |
|misstoriblack  |1    |
|globalstatmusic|1    |
|_fuckgio       |1    |
|PedroIvoChianca|1    |
|Cpiepz         |1    |
|avachristy3    |1    |
|lostbayouramble|1    |
|bellahadid     |1    |
|sawano_nZk's   |1    |
|marIboros      |1    |
|kochamjacksona |1    |
|WIOD           |2    |
|ShaelynCherie  |2    |
|KevinAnex      |1    |
+---------------+-----+
only showing top 20 rows



##5. Find the 10 most mentioned persons


In [39]:
new_usermentioned_df.groupBy('users_mentioned').count().orderBy('count', ascending=False).limit(10).show()

+---------------+-----+
|users_mentioned|count|
+---------------+-----+
|    ShawnMendes|  189|
|  HIITMANonDECK|  100|
|officialdjjuice|   59|
|         MAEJOR|   45|
|    MR_JAYJONES|   41|
|       MeekMill|   35|
|MadisonElleBeer|   30|
|              …|   28|
|     DjLordDash|   27|
|     NICKIMINAJ|   25|
+---------------+-----+



##6. Find all the hashtags mentioned on a tweet

In [43]:
def generate_hashtags(text):
    return [item for item in text.split(' ') if item.startswith('#')]
hashtags_mentioned = df_twitter.withColumn('hashtags_mentioned', F.udf(lambda text: generate_hashtags(text), ArrayType(StringType()))('text'))
hashtags_mentioned.show()

+--------------------+------------------+-----------------+--------------------+-------------------+--------------------+
|             country|                id|            place|                text|               user|  hashtags_mentioned|
+--------------------+------------------+-----------------+--------------------+-------------------+--------------------+
|               India|572692378957430785|           Orissa|@always_nidhi @Yo...|    Srkian_nishu :)|                  []|
|       United States|572575240615796737|        Manhattan|@OnlyDancers Bell...| TagineDiningGlobal|                  []|
|       United States|572575243883036672|        Claremont|1/ "Without the a...|        Daniel Beer|                  []|
|       United States|572575252020109313|           Vienna|idk why people ha...|   someone actually|                  []|
|       United States|572575274539356160|           Boston|Taste of Iceland!...|     BostonAttitude|                  []|
|       United States|57

##7. Count how many times each hashtag is mentioned

In [45]:
new_hashtags_mentioned= hashtags_mentioned.select(F.explode('hashtags_mentioned').alias('hashtags_mentioned'))
new_hashtags_mentioned = new_hashtags_mentioned.filter(new_hashtags_mentioned['hashtags_mentioned'] != '')
new_hashtags_mentioned.groupBy('hashtags_mentioned').count().show(truncate=False)

+--------------------+-----+
|hashtags_mentioned  |count|
+--------------------+-----+
|#2NE1               |3    |
|#musicLover         |1    |
|#IBMCloud           |2    |
|#flexrecordingstudio|1    |
|#Hottest            |1    |
|#VanessaBorn        |1    |
|#happychappy        |1    |
|#yyjevents          |1    |
|#LittleLionMan      |1    |
|#MBAMBADU           |7    |
|#misheardlyrics     |1    |
|#Indie              |2    |
|#family             |1    |
|#beautiful          |2    |
|#Waiter             |1    |
|#friend             |1    |
|#recuseimitaçoes    |1    |
|#airbnb             |1    |
|#BØRNS              |1    |
|#ChickCorea         |1    |
+--------------------+-----+
only showing top 20 rows



##8. Find the 10 most popular Hashtags

In [46]:
new_hashtags_mentioned.groupBy('hashtags_mentioned').count().orderBy('count', ascending=False).limit(10).show()

+-------------------+-----+
| hashtags_mentioned|count|
+-------------------+-----+
|               #DME|  253|
|          #ROADBOYZ|  251|
|             #music|  236|
|             #Paris|  144|
|#QuissyUpSoon🔥🔥💯|  129|
|      #QuissyUpSoon|  120|
| #Trippythursdaymia|  100|
|             #Music|   84|
|    #MaejorMeAndYou|   44|
|              #IGGL|   41|
+-------------------+-----+



##9.Find the top 5 countries which tweet the most.

In [13]:
df_twitter.groupBy('country').count().orderBy('count', ascending=False).limit(5).show()

+--------------+-----+
|       country|count|
+--------------+-----+
| United States| 4841|
|        France|  737|
|     Indonesia|  370|
|United Kingdom|  365|
|        Brasil|  256|
+--------------+-----+

